In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wistl.config import TransmissionConfig
from wistl.transmission_network import TransmissionNetwork
cfg = TransmissionConfig('./tests/test_line_interaction.cfg')
network = TransmissionNetwork(cfg)

In [3]:
network.event_tuple = ('test2', 3.5)

# looking at Calaca - Amadeo line, which interacts with C-S and A-C
## AC-100 -> CB-005 (C-S) and DA-001 (A-C)

In [4]:
DA001 = network.lines['Amadeox - Calacax'].towers['DA-001']
CB005 = network.lines['Calaca - Santa Rosa'].towers['CB-005']
AC100 = network.lines['Calaca - Amadeo'].towers['AC-100']

In [5]:
AC100.id_on_target_line

{'Amadeox - Calacax': {'id': 1, 'vector': array([-0.44018835,  0.89790546])},
 'Calaca - Santa Rosa': {'id': 4, 'vector': array([ 0.72409081, -0.68970465])}}

In [6]:
AC100.coord

array([ 120.80279139,   13.93656573])

## compute unit vectors

In [7]:
import numpy as np
def unit_vector(vector):
    """
    vector = tuple(x, y)
    returns unit vector
    """
    return vector / np.linalg.norm(vector)

In [8]:
unit_vector([1.0, 2.0])

array([ 0.4472136 ,  0.89442719])

In [9]:
def unit_vector_by_bearing(angle_deg):
    # agnle should be between 0 and 360
    angle_rad = np.deg2rad(angle_deg)
    return np.array([ -1.0*np.sin(angle_rad), -1.0*np.cos(angle_rad)])

In [10]:
AC100.wind.head()

,Speed,Bearing,dir_speed,ratio
Time,,,,
2014-07-13 09:00:00,12.600052,1.841016,12.877253,0.157879
2014-07-13 09:05:00,12.622552,1.841016,12.900248,0.158161
2014-07-13 09:10:00,12.645032,1.837737,12.923223,0.158442
2014-07-13 09:15:00,12.645012,1.834469,12.923203,0.158442
2014-07-13 09:20:00,12.667493,1.831213,12.946178,0.158724


In [11]:
wind_vector = unit_vector_by_bearing(271.0)
wind_vector

array([ 0.9998477 , -0.01745241])

In [12]:
def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0), (0, 1))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    #     v1_u = unit_vector(v1)
    #     v2_u = unit_vector(v2)
    return np.degrees(np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0)))

In [13]:
CB005toAC100 = AC100.id_on_target_line['Calaca - Santa Rosa']['vector']
angle_between(wind_vector, CB005toAC100)

42.606733678075436

In [14]:
DA001toAC100 = AC100.id_on_target_line['Amadeox - Calacax']['vector']
angle_between(wind_vector, DA001toAC100)

117.11589926696573

In [15]:
for angle_ in [1.0, 91.0, 181.0, 271.0, 355.0]:
    wind_vector = unit_vector_by_bearing(angle_)
    angle1 = angle_between(wind_vector, DA001toAC100)
    angle2 = angle_between(wind_vector, CB005toAC100)
    
    print('{:.2f}:{:.2f}:{:.2f}'.format(angle_, angle1, angle2))

1.00:152.88:47.39
91.00:62.88:137.39
181.00:27.12:132.61
271.00:117.12:42.61
355.00:158.88:41.39


In [16]:
angle_between((1, 0), (1, 0))

0.0

In [17]:
rnd_state = np.random.RandomState(11)
rv = rnd_state.uniform(size=(AC100.cfg.no_sims, len(AC100.time_index)))
AC100.determine_damage_isolation_mc(rv)

0.0:0.0377700333937
0.2:0.31994710948
0.5:0.45873553118
0.5:0.617590659514
0.9:0.761418465102
0.35:0.450655979037
0.05:0.110418062933
0.2:0.135965215778
0.6:0.540116911949
0.85:0.929917045518
0.95:0.869587809994
0.6:0.542474690508
0.1:0.1810079591
0.5:0.459733454979
0.0:0.0147786484814
0.1:0.0739051902147
0.15:0.129551638904
0.3:0.20125929653
0.15:0.303308477684
0.75:0.546522699541
0.6:0.673188328377
0.7:0.762772786077
0.95:0.846781056714
1.0:0.911921262675
0.75:0.826495953901
0.25:0.196707919951
0.0:0.0414872591689
0.1:0.051219985122
0.2:0.250265926625
1.0:0.931825886587
0.85:0.909463784399
0.65:0.798505507762
0.65:0.557770915927
0.4:0.251779764485
0.05:0.0689760959997
0.35:0.201825084534


In [18]:
AC100.damage_isolation_mc['collapse']

,id_sim,id_time
0,0,713
1,0,714
2,0,715
3,0,716
4,0,717
5,0,718
6,0,719
7,0,720
8,0,721
9,0,722


In [20]:
no_sim_collapse = len(AC100.damage_isolation_mc['collapse']['id_time'])
AC100.determine_damage_by_interaction(11)
AC100.damage_interaction_mc.head()

,id_sim,id_time,no_collapse
1,0,714,1
5,0,718,1
8,0,721,5
10,0,723,1
17,0,730,1


In [21]:
for line_, value_ in AC100.id_on_target_line.iteritems():
    print('{}:{}'.format(line_, value_))
    AC100.id_on_target_line

Amadeox - Calacax:{'vector': array([-0.44018835,  0.89790546]), 'id': 1}
Calaca - Santa Rosa:{'vector': array([ 0.72409081, -0.68970465]), 'id': 4}


In [26]:
line = network.lines['Calaca - Amadeo']

tf_ds = dict()
for target_line in line.cfg.line_interaction['Calaca - Amadeo']:
    tf_ds[target_line] = np.zeros((line.cfg.no_towers_by_line[target_line],
                                 line.cfg.no_sims, 
                                 len(line.time_index)), dtype=bool)

In [32]:
target_tower_id = 1
max_no_towers = 6
no_towers_on_either_side = 5

list_one_direction = AC100.create_list_idx(target_tower_id, no_towers_on_either_side, max_no_towers, +1)
print ('{}'.format(list_one_direction))

list_another_direction = AC100.create_list_idx(target_tower_id, no_towers_on_either_side, max_no_towers, -1)
print ('{}'.format(list_another_direction))

list_towers = [x for x in list_one_direction + [target_tower_id] + list_another_direction if x >=0]

print ('{}'.format(list_towers))

[2, 3, 4, 5, -1]
[0, -1, -1, -1, -1]
[2, 3, 4, 5, 1, 0]


In [38]:
grouped['no_collapse']

159    1
263    5
Name: no_collapse, dtype: float64

In [41]:
for id_time, grouped in AC100.damage_interaction_mc.groupby('id_time'):
    for no_collapse, sub_grouped in grouped.groupby('no_collapse'):
        print('{}:{}:{}'.format(id_time, no_collapse, sub_grouped.head()))

708:1.0:    id_sim  id_time  no_collapse
50       2      708            1
709:1.0:     id_sim  id_time  no_collapse
159       6      709            1
709:5.0:     id_sim  id_time  no_collapse
263      10      709            5
710:3.0:     id_sim  id_time  no_collapse
103       4      710            3
426      16      710            3
711:1.0:     id_sim  id_time  no_collapse
264      10      711            1
712:1.0:     id_sim  id_time  no_collapse
237       9      712            1
400      15      712            1
712:3.0:     id_sim  id_time  no_collapse
346      13      712            3
713:1.0:     id_sim  id_time  no_collapse
78        3      713            1
105       4      713            1
160       6      713            1
713:3.0:     id_sim  id_time  no_collapse
238       9      713            3
714:1.0:     id_sim  id_time  no_collapse
1         0      714            1
239       9      714            1
266      10      714            1
348      13      714            1
714:

In [47]:
for (id_time, no_collapse), grouped in AC100.damage_interaction_mc.groupby(['id_time', 'no_collapse']):
    print('{}:{}'.format(id_time, no_collapse))

708:1.0
709:1.0
709:5.0
710:3.0
711:1.0
712:1.0
712:3.0
713:1.0
713:3.0
714:1.0
714:3.0
715:1.0
715:3.0
716:1.0
716:3.0
717:1.0
717:3.0
718:1.0
718:3.0
718:5.0
719:1.0
719:3.0
719:5.0
720:1.0
720:3.0
720:5.0
721:1.0
721:3.0
721:5.0
722:1.0
722:3.0
723:1.0
723:3.0
724:1.0
724:3.0
725:1.0
725:3.0
726:1.0
726:3.0
726:5.0
727:1.0
727:3.0
728:1.0
729:1.0
729:3.0
730:1.0
730:3.0
731:1.0
731:3.0
732:1.0
732:3.0
733:1.0
733:3.0
734:1.0
734:3.0
736:1.0
737:3.0
738:1.0
738:3.0
739:1.0
739:3.0
739:5.0
740:1.0
741:1.0
741:3.0
742:1.0
742:3.0
742:5.0
743:1.0
743:3.0
744:1.0
746:1.0
746:5.0


In [50]:
import itertools

In [52]:
for id_time, grouped in AC100.damage_interaction_mc.groupby('id_time'):
    wind_vector = unit_vector_by_bearing(AC100.wind['Bearing'][id_time])
    
    angle = dict()
    for line_name, value in AC100.id_on_target_line.iteritems():
        angle[line_name] = angle_between(wind_vector, value['vector'])
        
    line_to_be_fallen = min(angle, key=angle.get)
    
    if angle[line_to_be_fallen] < 180:

        target_tower_id = AC100.id_on_target_line[line_to_be_fallen]['id']
        max_no_towers = AC100.cfg.no_towers_by_line[line_to_be_fallen]

        for no_collapse, sub_grouped in grouped.groupby('no_collapse'):
        
            no_towers_on_either_side = int(no_collapse/2)

            list_one_direction = AC100.create_list_idx(target_tower_id, no_towers_on_either_side, max_no_towers, +1)
            list_another_direction = AC100.create_list_idx(target_tower_id, no_towers_on_either_side, max_no_towers, -1)
        
            list_towers = [x for x in list_one_direction + [target_tower_id] + list_another_direction if x >=0]
        
            print('{}:{}:{}'.format(id_time, no_collapse, list_towers))
            for r in itertools.product(list_towers, sub_grouped['id_sim'].values):
                tf_ds[line_to_be_fallen][r[0], r[1], id_time] = True
    else:
        print ('It does not make sense')

708:1.0:[4]
709:1.0:[4]
709:5.0:[5, 4, 3, 2]
710:3.0:[5, 4, 3]
711:1.0:[4]
712:1.0:[4]
712:3.0:[5, 4, 3]
713:1.0:[4]
713:3.0:[5, 4, 3]
714:1.0:[4]
714:3.0:[5, 4, 3]
715:1.0:[4]
715:3.0:[5, 4, 3]
716:1.0:[4]
716:3.0:[5, 4, 3]
717:1.0:[4]
717:3.0:[5, 4, 3]
718:1.0:[4]
718:3.0:[5, 4, 3]
718:5.0:[5, 4, 3, 2]
719:1.0:[4]
719:3.0:[5, 4, 3]
719:5.0:[5, 4, 3, 2]
720:1.0:[4]
720:3.0:[5, 4, 3]
720:5.0:[5, 4, 3, 2]
721:1.0:[4]
721:3.0:[5, 4, 3]
721:5.0:[5, 4, 3, 2]
722:1.0:[4]
722:3.0:[5, 4, 3]
723:1.0:[4]
723:3.0:[5, 4, 3]
724:1.0:[4]
724:3.0:[5, 4, 3]
725:1.0:[4]
725:3.0:[5, 4, 3]
726:1.0:[4]
726:3.0:[5, 4, 3]
726:5.0:[5, 4, 3, 2]
727:1.0:[4]
727:3.0:[5, 4, 3]
728:1.0:[4]
729:1.0:[4]
729:3.0:[5, 4, 3]
730:1.0:[4]
730:3.0:[5, 4, 3]
731:1.0:[4]
731:3.0:[5, 4, 3]
732:1.0:[4]
732:3.0:[5, 4, 3]
733:1.0:[4]
733:3.0:[5, 4, 3]
734:1.0:[4]
734:3.0:[5, 4, 3]
736:1.0:[4]
737:3.0:[5, 4, 3]
738:1.0:[4]
738:3.0:[5, 4, 3]
739:1.0:[4]
739:3.0:[5, 4, 3]
739:5.0:[5, 4, 3, 2]
740:1.0:[4]
741:1.0:[4]
741:3.0:[5, 4

In [146]:
AC100.damage_interaction_mc.head()

,id_sim,id_time,no_collapse
1,0,714,1
5,0,718,1
8,0,721,5
10,0,723,1
17,0,730,1


In [141]:
AC100.damage_isolation_mc['collapse']['id_sim'].values

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8